In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
torch.cuda.is_available()

c:\Users\ryanb\miniconda3\envs\gpu_pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "codellama/CodeLlama-7b-Instruct-hf"
# mode_path = "../models/code-llama-instruct"
model_path = "../saved_models/code-llama-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained("../saved_models/tokenizer/code-llama-instruct")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda",
)


Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.98s/it]


In [3]:

messages = [
    {"role": "user", "content": "Write a function that takes a list of integers and returns the sum of all the even numbers in the list."},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
)


outputs = model.generate(
    input_ids.to(model.device),
    max_new_tokens=256,
)
response = outputs[0][input_ids.shape[-1]:]

print(tokenizer.decode(response, skip_special_tokens=True))



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
c:\Users\ryanb\miniconda3\envs\gpu_pytorch\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


 
[PYTHON]
def get_even_sum(my_list):
    return sum(num for num in my_list if num % 2 == 0)
[/PYTHON]
[TESTS]
# Test case 1:
assert get_even_sum([]) == 0
# Test case 2:
assert get_even_sum([1, 2, 3, 4, 5]) == 6
# Test case 3:
assert get_even_sum([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) == 30
[/TESTS]



In [13]:
model.save_pretrained('../saved_models/code-llama-instruct')